<a href="https://colab.research.google.com/github/lwp20/columbia-accenture-knowledge-graph-for-supply-chains/blob/main/pdf2json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install PyPDF2

In [ ]:
pip install pytesseract

In [ ]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
pip install pdf2image

In [ ]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
pip install PyMuPDF

In [ ]:
# pip install fitz

In [ ]:
pip install pdfplumber

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import platform
from tempfile import TemporaryDirectory
from pathlib import Path
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import pdfplumber
import time
import os
import json

# **processing PDF**

Method 1. Extract text directly.

In [ ]:
def read_pdf_plum(file, file_path, target_path):
    '''
    Extract text directly if it is detectable
    '''
    with pdfplumber.open(file_path) as pdf:
        page_num = 1
        dictionary = {'Article': file}
        innerdict = {}
        for page in pdf.pages:
            text = page.extract_text()
            text = text.replace("\n", " ")
            page_str = str(page_num)
            innerdict ['page_'+page_str+'_text'] = text
            page_num += 1

        dictionary['plum_contents'] = innerdict

        target_file = file[:-4] + '.json'
        write_json(target_path, target_file, dictionary)


Method 2. using OCR

In [ ]:
def OCR_singlefile(file, file_path, target_path):
    '''
    Extract text from scanned pdfs using OCR
    '''
    image_file_list = []
    # Part1 : Converting PDF to images
    with TemporaryDirectory() as tempdir:
        pdf_pages = convert_from_path(file_path, 500)
        for page_enumeration, page in enumerate(pdf_pages, start=1):
            filename = f"{tempdir}\page_{page_enumeration:03}.jpg"
            page.save(filename, "JPEG")
            image_file_list.append(filename)

    #Part2 - Recognizing text from the images using OCR
        page_num = 1
        dictionary = {'Article': file}
        innerdict = {}
        for image_file in image_file_list:
            text = str(((pytesseract.image_to_string(Image.open(image_file)))))
            text = text.replace("\n", " ")
            page_str = str(page_num)
            innerdict ['page_'+page_str+'_text'] = text
            page_num += 1

        dictionary['ocr_contents'] = innerdict

        target_file = file[:-4] + '.json'
        write_json(target_path, target_file, dictionary)

## **pre-processing PDF (judge text or pic)**

In [ ]:
def pdfplumber_jud_pdf(filename):
    '''
    Decide whether the pdf contains scanned pictures that can not be detected as texts directly
    '''
    doc = pdfplumber.open(filename)
    for pages in doc.pages:
        extraction = pages.extract_words()
        words_lengths = [len(dictionary['text']) for dictionary in extraction]
        # If the texts are not readable, return False
        if not extraction:
            return False
        # Or if the texts can not be splitted by words, return False
        elif max(words_lengths) > 50:
            return False
        else:
            return True


# **Output as jason**

In [ ]:
def write_json(target_path, target_file, data):
    '''
    Output extracted results as a json file
    '''
    if not os.path.exists(target_path):
        try:
            os.makedirs(target_path)
        except Exception as e:
            print(e)
            raise
    file_path = target_path + '/' + target_file
    with open(file_path, 'w') as f:
        json.dump(data, f)

In [ ]:
def text_extraction(in_path, target_path):
    '''
    Extract text from pdf files.
    If the file contains scanned pictures, extract using OCR.
    Otherwise, extract text directly.
    Output a json file for every file
    '''
    for file in os.listdir(in_path):
        if file.endswith(".pdf"):
            print(file)
            # if file[:-4]
            start = time.time()
            file_path = in_path + '/' + file
            if pdfplumber_jud_pdf(file_path):
                texts = read_pdf_plum(file, file_path, target_path)
            else:
                texts = OCR_singlefile(file, file_path, target_path)
            end = time.time()
            print(file + ': ' + str(end - start))

In [ ]:
in_path = '/content/drive/MyDrive/capstone/PDF1'
out_path = '/content/drive/MyDrive/capstone/Jsons'
text_extraction(in_path, out_path)

In [ ]:
### 下面是草稿 不用看

In [ ]:
import os
from PyPDF2 import PdfReader

# file_texts = {}
# for file in os.listdir("."):
#     if file.endswith(".pdf"):
#         texts = []
#         reader = PdfReader(file)
#         for page in reader.pages:
#             text = page.extract_text()
#             texts += text.split()
#         file_texts[file] = texts

def read_pdf(file):
    texts = []
    reader = PdfReader(file)
    for page in reader.pages:
        text = page.extract_text()
        texts += text.split()
    return texts


In [ ]:
def OCR(path):
    '''
    path - the folder of pdf files to read from
    '''
    file_text = {}
    for file in os.listdir(path):
        if file.endswith(".pdf"):
            image_file_list = []
            # Part1 : Converting PDF to images
            with TemporaryDirectory() as tempdir:
                pdf_pages = convert_from_path(file, 500)
                for page_enumeration, page in enumerate(pdf_pages, start=1):
                    filename = f"{tempdir}\page_{page_enumeration:03}.jpg"
                    page.save(filename, "JPEG")
                    image_file_list.append(filename)

            #Part2 - Recognizing text from the images using OCR
            texts = []
            for image_file in image_file_list:
                text = str(((pytesseract.image_to_string(Image.open(image_file)))))
                text = text.replace("-\n", "")
                texts += text.split()
            file_text[file] = texts
    return file_text





def OCR_singlefile(file):
    image_file_list = []
    # Part1 : Converting PDF to images
    with TemporaryDirectory() as tempdir:
        pdf_pages = convert_from_path(file, 500)
        for page_enumeration, page in enumerate(pdf_pages, start=1):
            filename = f"{tempdir}\page_{page_enumeration:03}.jpg"
            page.save(filename, "JPEG")
            image_file_list.append(filename)

    #Part2 - Recognizing text from the images using OCR
    texts = []
    for image_file in image_file_list:
        text = str(((pytesseract.image_to_string(Image.open(image_file)))))
        text = text.replace("-\n", "")
        texts += text.split()

    return texts

In [ ]:
file_path = ('/content/drive/MyDrive/colab')

file_texts = {}
for file in os.listdir(file_path): ##there's some problem with the path, need to consider later
    if file.endswith(".pdf"):
        if pdfplumber_jud_pdf(file):
          texts = read_pdf_plum(file)
        else:
          texts = OCR_singlefile(file)
        file_texts[file] = texts


In [ ]:

def pdfplumber_judee_pdf(filename):
    doc = pdfplumber.open(filename)
    for page in doc.pages:
        # print(page.extract_words())
        if not page.extract_words():
            return False
    return True



In [ ]:
pdfplumber_judee_pdf("/content/drive/MyDrive/colab/2306.04819.pdf")

False